In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
from datetime import datetime
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import EarthLocation
from astropy.coordinates import builtin_frames

In [4]:
data = [0]*(199+201)
for i in range(0,199):
    with h5py.File('/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/20201118/dl2_LST-1.Run02929.{:04d}.h5'
               .format(i), 'r') as f:
        data[i] = pd.DataFrame(np.array(f['dl2/event/telescope/parameters/LST_LSTCam']))
        
for i in range(0,201):
    with h5py.File('/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/20201118/dl2_LST-1.Run02930.{:04d}.h5'
               .format(i), 'r') as f:
         data[199+i] = pd.DataFrame(np.array(f['dl2/event/telescope/parameters/LST_LSTCam']))
            
tot_data = pd.concat([data[i] for i in range(0,199+201)])
tot_data.head()

,intensity,log_intensity,x,y,r,phi,length,width,psi,skewness,...,log_reco_energy,reco_energy,reco_disp_dx,reco_disp_dy,reco_src_x,reco_src_y,reco_alt,reco_az,reco_type,gammaness
0,14019.949485,4.146746,-0.015204,-0.005036,0.016016,-2.821747,1.173447,1.142609,0.518989,0.023384,...,1.161159,14.493039,0.586236,0.238408,0.571033,0.233373,1.281348,1.902129,101,0.081857
1,76.737119,1.885005,-0.527041,-0.305895,0.609380,-2.615709,0.079877,0.073439,0.426368,0.060108,...,-1.459769,0.034692,-0.153171,-0.079658,-0.680212,-0.385553,1.236501,1.830964,101,0.386667
2,25.876774,1.412910,-0.349313,0.001244,0.349315,3.138032,0.088380,0.040477,0.769460,0.026443,...,-1.569809,0.026927,0.050654,0.022915,-0.298659,0.024159,1.250402,1.875670,101,0.305238
3,25.759664,1.410940,0.122801,-0.445033,0.461665,-1.301559,0.061317,0.053258,-1.157097,0.232783,...,-1.334617,0.046279,0.056081,-0.049218,0.178882,-0.494252,1.266962,1.813900,101,0.226262
4,44.028748,1.643736,-0.141567,0.933637,0.944309,1.721280,0.080193,0.045229,-1.103720,0.250541,...,-1.414461,0.038507,0.080160,0.050695,-0.061408,0.984333,1.256965,1.987032,101,0.251333


In [5]:
tot_data.shape

(18213591, 50)

In [ ]:
data = tot_data[(tot_data['gammaness']>0.8)]
data.shape

In [ ]:
data.columns

## Calculate the equatorial coordinates (right ascension and declination) of the events + make a skymap of the reconstructed directions of the events in these coordinates.

In [ ]:
data.head()

In [ ]:
utc_time = []
for i in range(data.shape[0]):
    a = datetime.utcfromtimestamp(data.iloc[i]['dragon_time'])
    utc_time.append(a)
    
data['utc_time'] = utc_time   #add new column to the data frame with the UTC observation times of the events

loc = EarthLocation(lat = 28.76138889*u.deg, lon = -17.89138889*u.deg, height = 2184*u.m)   #location of the telescope (CTA LST1: Roque de los Muchachos)

hor_coords = SkyCoord(alt = data['reco_alt'], az = data['reco_az'], frame = 'altaz', unit = 'rad', 
                      obstime = data['utc_time'], location = loc)   #horizontal coordinates of the events
eq_coords = hor_coords.icrs   #equatorial coordinates of the events

data['RA'] = eq_coords.ra    #add columns with right ascension and declination (equatorial coordinates) of the events
data['DEC'] = eq_coords.dec

# skymap (RA, dec) de las posiciones reconstruidas de los sucesos (ya se ha aplicado el corte de gammaness > 0.8):
plt.figure(figsize=(8,4))
h = plt.hist2d(data['RA'], data['DEC'], bins = 175, cmap = 'inferno')
plt.xlabel('right ascension (deg)')
plt.ylabel('declination (deg)')
plt.colorbar(label='counts')

## Ver que hay un exceso significativo de sucesos (gammas) en las coordenadas del Crab y estimar el número de gammas (haciendo "aperture photometry").

In [ ]:
print(max(data['intensity']))
print(min(data['intensity']))

In [ ]:
data1 = data[(data['intensity']>200)]
data1.shape

In [ ]:
crab = SkyCoord.from_name('M1')    #equatorial coordinates of the Crab nebula in deg

eq_coords = SkyCoord(data1['RA'], data1['DEC'], frame='icrs', unit='deg')   #frame 'icrs' = equatorial coordinates (of the events)
data1['theta²'] = (eq_coords.separation(crab))**2  #square of the angular separation between the events and the crab (in deg)

hor_coords_tel = SkyCoord(alt = data1['alt_tel'], az = data1['az_tel'], frame = 'altaz', unit = 'rad', 
                      obstime = data1['utc_time'], location = loc)   #horizontal coordinates of the telescope
eq_coords_tel = hor_coords_tel.icrs   #equatorial coordinates of the telescope

data1['RA_tel'] = eq_coords_tel.ra    #add new columns with the pointing direction of the telescope at each moment in equatorial coordinates (ra and dec)
data1['DEC_tel'] = eq_coords_tel.dec  

skyoffset_frame = builtin_frames.SkyOffsetFrame(origin = eq_coords_tel)  #define a new reference frame centered at the pointing direction of the telescope at each moment

crab2 = crab.transform_to(skyoffset_frame)   #position of the Crab nebula with respect to the new frame 

sym_pos = SkyCoord(-crab2.lon, -crab2.lat, frame = skyoffset_frame, unit = 'deg')  #symmetric position to the position of the Crab with respect to the center of the FOV
sym_pos = sym_pos.transform_to('icrs') 

data1['theta²_2'] = (eq_coords.separation(sym_pos))**2 #square of the angular separation between the events and the symmetric position to the crab with respect to the center of the FOV (in deg)

plt.figure()
(n1, b1, p1) = plt.hist(data1['theta²'], bins=750, histtype = 'step')
(n2, b2, p2) = plt.hist(data1['theta²_2'], bins=750, histtype = 'step')
plt.errorbar((b1[1:]+b1[:-1])/2, n1, fmt = 'none' , yerr=np.sqrt(n1), elinewidth=0.5, capsize=1)  #the bin contents are described by poissonian distributions, so the the error (std) is given by sqrt(n)
plt.errorbar((b2[1:]+b2[:-1])/2, n2, fmt = 'none' , yerr=np.sqrt(n2), elinewidth=0.5, capsize=1)
plt.xlabel(r'$\theta²$')
plt.ylabel('counts')
plt.xlim(0,2.5)
plt.savefig('theta2.png')

excess_gammas = n1[0]+n1[1]-n2[0]-n2[1]
print('Excess of gamma rays: {}'. format(excess_gammas))

## Do the same, but instead of taking only one off zone, take three (añadir 2 más en forma de cruz para tener más estadística para calcular el fondo).

In [ ]:
crab = SkyCoord.from_name('M1')    #equatorial coordinates of the Crab nebula in deg

eq_coords = SkyCoord(data1['RA'], data1['DEC'], frame='icrs', unit='deg')   #frame 'icrs' = equatorial coordinates (of the events)
data1['theta²'] = (eq_coords.separation(crab))**2  #square of the angular separation between the events and the crab (in deg)

hor_coords_tel = SkyCoord(alt = data1['alt_tel'], az = data1['az_tel'], frame = 'altaz', unit = 'rad', 
                      obstime = data1['utc_time'], location = loc)   #horizontal coordinates of the telescope
eq_coords_tel = hor_coords_tel.icrs   #equatorial coordinates of the telescope

data1['RA_tel'] = eq_coords_tel.ra    #add new columns with the pointing direction of the telescope at each moment in equatorial coordinates (ra and dec)
data1['DEC_tel'] = eq_coords_tel.dec  

skyoffset_frame = builtin_frames.SkyOffsetFrame(origin = eq_coords_tel)  #define a new reference frame centered at the pointing direction of the telescope at each moment

crab2 = crab.transform_to(skyoffset_frame)   #position of the Crab nebula with respect to the new frame 

pos1 = SkyCoord(-crab2.lon, -crab2.lat, frame = skyoffset_frame, unit = 'deg')  #symmetric position to the position of the Crab with respect to the center of the FOV
pos2 = SkyCoord(crab2.lon, -crab2.lat, frame = skyoffset_frame, unit = 'deg') 
pos3 = SkyCoord(-crab2.lon, crab2.lat, frame = skyoffset_frame, unit = 'deg') 
pos1 = pos1.transform_to('icrs') 
pos2 = pos2.transform_to('icrs')
pos3 = pos3.transform_to('icrs')

data1['theta²_2'] = (eq_coords.separation(pos1))**2 
data1['theta²_3'] = (eq_coords.separation(pos2))**2 
data1['theta²_4'] = (eq_coords.separation(pos3))**2

plt.figure()
(n1, b1, p1) = plt.hist(data1['theta²'], bins=750, histtype = 'step')
(n2, b2, p2) = plt.hist(data1['theta²_2'], bins=750, histtype = 'step')
(n3, b3, p3) = plt.hist(data1['theta²_3'], bins=750, histtype = 'step')
(n4, b4, p4) = plt.hist(data1['theta²_4'], bins=750, histtype = 'step')
plt.errorbar((b1[1:]+b1[:-1])/2, n1, fmt = 'none' , yerr=np.sqrt(n1), elinewidth=0.5, capsize=1)  
plt.errorbar((b2[1:]+b2[:-1])/2, n2, fmt = 'none' , yerr=np.sqrt(n2), elinewidth=0.5, capsize=1)
plt.errorbar((b3[1:]+b3[:-1])/2, n3, fmt = 'none' , yerr=np.sqrt(n3), elinewidth=0.5, capsize=1) 
plt.errorbar((b4[1:]+b4[:-1])/2, n4, fmt = 'none' , yerr=np.sqrt(n4), elinewidth=0.5, capsize=1)
plt.xlabel(r'$\theta²$')
plt.ylabel('counts')
plt.xlim(0, 2.5)
plt.savefig('theta2_2.png')

excess_gammas = n1[0]+n1[1]-np.mean([n2[0],n3[0],n4[0]])-np.mean([n2[1],n3[1],n4[1]])
print('Excess of gamma rays: {}'. format(excess_gammas))

## Obtain the distribution of theta², width, length and intensity of (only) the gamma rays.

#### *** ERROR PROPAGATION: 

($n_1$, $n_2$, $n_3$, $n_4$): histogram bins (for each position) 

($\sqrt{n_1}$, $\sqrt{n_2}$, $\sqrt{n_3}$, $\sqrt{n_4}$): errors of each "variable" ($n_i$)

new variable (function of $(n_1, n_2, n_3, n_4)$): $n(n_1, n_2, n_3, n_4)=n_1-\frac{1}{3}(n_2+n_3+n_4)$

Covariance matrix of the variables $n_i$ (since $\sigma_i=\sqrt{n_i}$ and $cov(x_i,x_i)=\sigma_i²$): 

\begin{equation*}
V = cov[n_i,n_j]=
\begin{pmatrix}
n_1 &  &  & \\
& n_2 &  & \\
 &  & n_3 &  \\
 &  &  &  n_4
\end{pmatrix}
\end{equation*}

To obtain the variance $U$ of the variable $n$, we use error propagation: 
$$U = \sum_{k,l=1}^n \frac{\partial n}{\partial n_i} \frac{\partial n}{\partial n_j} V_{kl}$$
$$A_1 = \frac{\partial n}{\partial n_1}=1  ; \quad A_2=A_3=A_4=-1/3 \quad \Rightarrow U=V_{11}+\frac{1}{9}(V_{22}+V_{33}+V_{44})$$

In [ ]:
gammas = data1[(data1['theta²']<0.5)]
bkg2 = data1[(data1['theta²_2']<0.5)]
bkg3 = data1[(data1['theta²_3']<0.5)]
bkg4 = data1[(data1['theta²_4']<0.5)]

In [ ]:
# distribution of theta² of the gamma rays

a = min(data1['theta²'])
b = max(data1['theta²'])
plt.figure()
(n1,b1,p1) = plt.hist(gammas['theta²'], histtype = 'step', bins = 100, range = (a,b))
(n2,b2,p2) = plt.hist(bkg2['theta²'], histtype = 'step', bins = 100, range = (a,b))
(n3,b3,p3) = plt.hist(bkg3['theta²'], histtype = 'step', bins = 100, range = (a,b))
(n4,b4,p4) = plt.hist(bkg4['theta²'], histtype = 'step', bins = 100, range = (a,b))
plt.errorbar((b1[1:]+b1[:-1])/2, n1, fmt = 'none' , yerr=np.sqrt(n1), elinewidth=0.5, capsize=1)  
plt.errorbar((b2[1:]+b2[:-1])/2, n2, fmt = 'none' , yerr=np.sqrt(n2), elinewidth=0.5, capsize=1)
plt.errorbar((b3[1:]+b3[:-1])/2, n3, fmt = 'none' , yerr=np.sqrt(n3), elinewidth=0.5, capsize=1) 
plt.errorbar((b4[1:]+b4[:-1])/2, n4, fmt = 'none' , yerr=np.sqrt(n4), elinewidth=0.5, capsize=1)
plt.xlabel('theta²')
plt.ylabel('counts')

n = n1-(1/3)*(n2+n3+n4)
err_n = np.sqrt(n1+1/9*(n2+n3+n4))  #error of n, obtained by error propagation 
#for i in range(len(n)): 
 #   if n[i]<0:
  #      n[i]=0

width = b1[1] - b1[0]
plt.figure()
plt.bar(b1[:-1], n, align='edge', width=width)
plt.errorbar((b1[1:]+b1[:-1])/2, n, fmt = 'none', ecolor='k', yerr=err_n, elinewidth=0.5, capsize=1)
plt.xlabel('theta²')
plt.ylabel('counts')
#plt.xlim(0,2.5)

In [ ]:
# distribution of the width of the cascades initiated by gamma rays

a = min(data1['width'])
b = max(data1['width'])
plt.figure()
(n1,b1,p1) = plt.hist(gammas['width'], histtype = 'step', bins = 100, range = (a,b))
(n2,b2,p2) = plt.hist(bkg2['width'], histtype = 'step', bins = 100, range = (a,b))
(n3,b3,p3) = plt.hist(bkg3['width'], histtype = 'step', bins = 100, range = (a,b))
(n4,b4,p4) = plt.hist(bkg4['width'], histtype = 'step', bins = 100, range = (a,b))
plt.errorbar((b1[1:]+b1[:-1])/2, n1, fmt = 'none' , yerr=np.sqrt(n1), elinewidth=0.5, capsize=1)  
plt.errorbar((b2[1:]+b2[:-1])/2, n2, fmt = 'none' , yerr=np.sqrt(n2), elinewidth=0.5, capsize=1)
plt.errorbar((b3[1:]+b3[:-1])/2, n3, fmt = 'none' , yerr=np.sqrt(n3), elinewidth=0.5, capsize=1) 
plt.errorbar((b4[1:]+b4[:-1])/2, n4, fmt = 'none' , yerr=np.sqrt(n4), elinewidth=0.5, capsize=1)
plt.xlabel('width')
plt.ylabel('counts')

n = n1-(1/3)*(n2+n3+n4)
err_n = np.sqrt(n1+1/9*(n2+n3+n4))  #error of n, obtained by error propagation

width = b1[1] - b1[0]
plt.figure()
plt.bar(b1[:-1], n, align='edge', width=width)
plt.errorbar((b1[1:]+b1[:-1])/2, n, fmt = 'none', ecolor='k', yerr=err_n, elinewidth=0.5, capsize=1)
plt.xlabel('width')
plt.ylabel('counts')


#plot de 'width' en bins de intensity: 

df1 = gammas[(gammas['width']>=b1[0]) & (gammas['width']<=b1[1])]
df2 = bkg2[(bkg2['width']>=b1[0]) & (bkg2['width']<=b1[1])]
df3 = bkg3[(bkg3['width']>=b1[0]) & (bkg3['width']<=b1[1])]
df4 = bkg4[(bkg4['width']>=b1[0]) & (bkg4['width']<=b1[1])]
n2 = [df1.sum(axis=0)['intensity']-(1/3)*(df2.sum(axis=0)['intensity']+df3.sum(axis=0)['intensity']+
                                          df4.sum(axis=0)['intensity'])]    #intensity
for i in range(1,len(b1)-1):
    df1 = gammas[(gammas['width']>b1[i]) & (gammas['width']<=b1[i+1])]
    n1 = [df1.sum(axis=0)['intensity']-(1/3)*(df2.sum(axis=0)['intensity']+df3.sum(axis=0)['intensity']+
                                          df4.sum(axis=0)['intensity'])]    #intensity
    n2.append(n1)

plt.figure()
plt.bar(b1[:-1], n2, align='edge', width=b1[0]-b1[1])
plt.xlabel('width')
plt.ylabel('intensity')

In [ ]:
# distribution of the length of the cascades initiated by gamma rays

a = min(data1['length'])
b = max(data1['length'])
plt.figure()
(n1,b1,p1) = plt.hist(gammas['length'], histtype = 'step', bins = 100, range = (a,b))
(n2,b2,p2) = plt.hist(bkg2['length'], histtype = 'step', bins = 100, range = (a,b))
(n3,b3,p3) = plt.hist(bkg3['length'], histtype = 'step', bins = 100, range = (a,b))
(n4,b4,p4) = plt.hist(bkg4['length'], histtype = 'step', bins = 100, range = (a,b))
plt.errorbar((b1[1:]+b1[:-1])/2, n1, fmt = 'none' , yerr=np.sqrt(n1), elinewidth=0.5, capsize=1)  
plt.errorbar((b2[1:]+b2[:-1])/2, n2, fmt = 'none' , yerr=np.sqrt(n2), elinewidth=0.5, capsize=1)
plt.errorbar((b3[1:]+b3[:-1])/2, n3, fmt = 'none' , yerr=np.sqrt(n3), elinewidth=0.5, capsize=1) 
plt.errorbar((b4[1:]+b4[:-1])/2, n4, fmt = 'none' , yerr=np.sqrt(n4), elinewidth=0.5, capsize=1)
plt.xlabel('length')
plt.ylabel('counts')

n = n1-(1/3)*(n2+n3+n4)
err_n = np.sqrt(n1+1/9*(n2+n3+n4))  #error of n, obtained by error propagation

width = b1[1] - b1[0]
plt.figure()
plt.bar(b1[:-1], n, align='edge', width=width)
plt.errorbar((b1[1:]+b1[:-1])/2, n, fmt = 'none', ecolor='k', yerr=err_n, elinewidth=0.5, capsize=1)
plt.xlabel('length')
plt.ylabel('counts')

In [ ]:
# distribution of log_intensity of the cascades initiated by gamma rays

a = min(data1['log_intensity'])
b = max(data1['log_intensity'])
plt.figure()
(n1,b1,p1) = plt.hist(gammas['log_intensity'], histtype = 'step', bins = 100, range = (a,b))
(n2,b2,p2) = plt.hist(bkg2['log_intensity'], histtype = 'step', bins = 100, range = (a,b))
(n3,b3,p3) = plt.hist(bkg3['log_intensity'], histtype = 'step', bins = 100, range = (a,b))
(n4,b4,p4) = plt.hist(bkg4['log_intensity'], histtype = 'step', bins = 100, range = (a,b))
plt.errorbar((b1[1:]+b1[:-1])/2, n1, fmt = 'none' , yerr=np.sqrt(n1), elinewidth=0.5, capsize=1)  
plt.errorbar((b2[1:]+b2[:-1])/2, n2, fmt = 'none' , yerr=np.sqrt(n2), elinewidth=0.5, capsize=1)
plt.errorbar((b3[1:]+b3[:-1])/2, n3, fmt = 'none' , yerr=np.sqrt(n3), elinewidth=0.5, capsize=1) 
plt.errorbar((b4[1:]+b4[:-1])/2, n4, fmt = 'none' , yerr=np.sqrt(n4), elinewidth=0.5, capsize=1)
plt.xlabel('log_intensity')
plt.ylabel('counts')

n = n1-(1/3)*(n2+n3+n4)
err_n = np.sqrt(n1+1/9*(n2+n3+n4))  #error of n, obtained by error propagation

width = b1[1] - b1[0]
plt.figure()
plt.bar(b1[:-1], n, align='edge', width=width)
plt.errorbar((b1[1:]+b1[:-1])/2, n, fmt = 'none', ecolor='k', yerr=err_n, elinewidth=0.5, capsize=1)
plt.xlabel('log_intensity')
plt.ylabel('counts')

In [ ]:
#EJEMPLO para hacer plots de distintos parámetros (en este caso width) en bins de intensity: 

d = {'intensity': [15, 2, 17, 28, 16, 17], 'width': [2, 2, 2, 1, 3, 4]}
df = pd.DataFrame(data=d)

a = min(df['width'])
b = max(df['width'])
(n1,b1,p1) = plt.hist(df['width'], histtype = 'step', bins = 100, range = (a,b))

df1 = df[(df['width']>=b1[0]) & (df['width']<b1[1])]
n = [df1.sum(axis=0)['intensity']]    #intensity
for i in range(1,len(b1)-1):
    df1 = df[(df['width']>b1[i]) & (df['width']<=b1[i+1])]
    n1 = df1.sum(axis=0)['intensity']    #intensity
    n.append(n1)

plt.figure()
plt.bar(b1[:-1], n, align='edge')